In [1]:
from os import getcwd, path, environ
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

BASE_PATH = getcwd()
sys.path.append(BASE_PATH)

environ['NUM_WORKERS'] = '0'
# environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
data_imdb = pd.read_json('data/imdb/train.json')
data_binary_sst = pd.read_csv('data/binary_sst/train.csv')
# data_semeval = pd.read_csv('data/semeval/train.csv')
data = pd.concat([data_imdb, data_binary_sst], ignore_index=True)

val_data_imdb = pd.read_json('data/imdb/test.json')
val_data_binary_sst = pd.read_csv('data/binary_sst/test.csv')
# data_semeval = pd.read_csv('data/semeval/train.csv')
val_data = pd.concat([val_data_imdb, val_data_binary_sst], ignore_index=True)

data.head()

/home/luungoc2005/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.
/home/luungoc2005/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,id,label,sentence
0,57447.0,1,This was on at 2 or so In the morning one Satu...
1,42373.0,0,JUDAAI was a bold film by Raj Kanwar at it's t...
2,1158910.0,1,"Man, I loved this movie! This really takes me ..."
3,8571.0,0,Many King fans hate this because it departed f...
4,120003.0,0,The only possible way to enjoy this flick is t...


In [3]:
data.loc[data['label'] == 0, 'label'] = 'negative'
data.loc[data['label'] == 1, 'label'] = 'positive'

val_data.loc[val_data['label'] == 0, 'label'] = 'negative'
val_data.loc[val_data['label'] == 1, 'label'] = 'positive'

data.head()

,id,label,sentence
0,57447.0,positive,This was on at 2 or so In the morning one Satu...
1,42373.0,negative,JUDAAI was a bold film by Raj Kanwar at it's t...
2,1158910.0,positive,"Man, I loved this movie! This really takes me ..."
3,8571.0,negative,Many King fans hate this because it departed f...
4,120003.0,negative,The only possible way to enjoy this flick is t...


In [4]:
data['sentence'] = data['sentence'].str.replace('\n', ' ').replace('<br />', ' ').replace('  ', ' ')

In [5]:
from sent_to_vec.masked_lm.bert_model import BertLMWrapper
from sent_to_vec.masked_lm.train import LanguageModelLearner
from sent_to_vec.masked_lm.data import WikiTextDataset

from common.modules import BertAdam
from common.callbacks import PrintLoggerCallback, EarlyStoppingCallback, ModelCheckpointCallback, TensorboardCallback, ReduceLROnPlateau

Default language for this instance: en


In [6]:
model = BertLMWrapper(from_fp='bert_en_base.bin')
model.init_model()

dataset = WikiTextDataset()
dataset.initialize(model, data_texts=list(data['sentence']))

Featurizer previously fitted, continuing
Found 1732377 tokens


In [7]:
BATCH_SIZE = 16
n_epochs=5

learner = LanguageModelLearner(model,
    optimizer_fn=BertAdam,
    optimizer_kwargs={
        'lr': 2e-5,
        't_total': n_epochs * (len(dataset) // BATCH_SIZE),
        'warmup': 0.04
    })

Training in BERT mode


In [8]:
import torch
learner.fit(
    training_data=dataset,
    batch_size=BATCH_SIZE,
    epochs=n_epochs,
    callbacks=[
        PrintLoggerCallback(log_every=1, metrics=['loss']),
    ],
    gradient_accumulation_steps=10,
    clip_grad=1.0,
)


Gradient accumulation is supported by this class
Number of tokens 36000
BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(36000, 576, padding_idx=0)
      (position_embeddings): Embedding(104, 576, padding_idx=0)
      (token_type_embeddings): Embedding(2, 576, padding_idx=0)
      (LayerNorm): FusedLayerNorm(torch.Size([576]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.15)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=576, out_features=576, bias=True)
              (key): Linear(in_features=576, out_features=576, bias=True)
              (value): Linear(in_features=576, out_features=576, bias=True)
              (dropout): Dropout(p=0.15)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=576, out_features=576

In [ ]:
# model.save('bert_vi_sentiment_lm.bin')

In [ ]:
from text_classification.with_pretrained.model import LMClassifierWrapper
from text_classification.with_pretrained.train import LMClassifierLearner

In [ ]:
n_epochs=30
classifier = LMClassifierWrapper(encoder=model)
classifier_learner = LMClassifierLearner(
    classifier,
    optimizer_fn=BertAdam,
    optimizer_kwargs={
        'lr': 2e-5,
        't_total': n_epochs * (len(dataset) // BATCH_SIZE),
        'warmup': 0.04
    }
)

In [ ]:
# classifier.model.encoder

In [ ]:
classifier_learner.fit(
    training_data=(data['sentence'], data['label']),
    validation_data=(val_data['sentence'], val_data['label']),
    batch_size=BATCH_SIZE,
    epochs=n_epochs, 
    callbacks=[
        PrintLoggerCallback(log_every=1),
        ModelCheckpointCallback()
    ]
)

{'num_words': 36000, 'hidden_size': 576, 'num_hidden_layers': 6, 'num_attention_heads': 12, 'intermediate_size': 1200, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.15, 'attention_probs_dropout_prob': 0.15, 'max_position_embeddings': 104, 'featurizer_seq_len': 104, 'type_vocab_size': 2, 'initializer_range': 0.025, 'use_adasoft': True, 'append_sos_eos': True, 'featurizer_reserved_tokens': ['<START>', '<STOP>', '<UNK>', '<MASK>'], 'tokenize_fn': <function word_tokenize at 0x7f7b8798fc80>, 'input_shape': (None,), 'adasoft_cutoffs': [2000, 4000, 10000]}


/media/luungoc2005/Data/Projects/botbot-nlp/common/wrappers.py:582: UserWarning: Error orcurred in multiprocessing.set_start_method
  warnings.warn('Error orcurred in multiprocessing.set_start_method')


3m 31s (- 102m 26s) (1 3%) - loss: 0.6204 - accuracy: 0.6740
{'accuracy': tensor(0.6875, device='cuda:0'), 'loss': 0.5257763266563416}
1995
TEST:  - val_accuracy: 0.0002
Model Checkpoint: Saving checkpoint: _ 3m 31s _epoch 1_1995 loss_0_6204 accuracy_0_6740.bin


In [ ]:
test_data_imdb = pd.read_json('data/imdb/test.json')
test_data_binary_sst = pd.read_csv('data/binary_sst/test.csv')
# data_semeval = pd.read_csv('data/semeval/train.csv')
test_data = pd.concat([test_data_imdb, test_data_binary_sst], ignore_index=True)

In [ ]:
test_data.head()

In [ ]:
import torch
test_item = [test_data['sentence'][5]]
classifier(test_item, return_logits=True)[0]

In [ ]:
classifier.save('bert_en_sentiment.bin')

In [ ]:
raw_model = classifier.model

In [ ]:
classifier.label_encoder.classes_.tolist()

In [ ]:
classifier(test_item)

In [ ]:
# from common.torch_utils import to_gpu

# _, seq_tokens = raw_model.encoder(
#     to_gpu(classifier.featurizer.transform(test_item))
# )
# sequence_output = raw_model.rnn(seq_tokens)[0]
# print(sequence_output)

# output, idxs = torch.max(sequence_output, 0)
# print(idxs)
# idxs = idxs.data.cpu().numpy()

# sent = classifier.featurizer.transform(test_item)
# raw_sent = classifier.featurizer.inverse_transform(sent)
# print(raw_sent)

# import matplotlib.pyplot as plt

# argmaxs = [np.sum((idxs == k)) for k in range(len(sent[0]))]
# # argmaxs[0] = 1e-8
# print(argmaxs)
# x = range(len(sent[0]))
# y = [100.0 * n / np.sum(argmaxs) for n in argmaxs]
# print(y)

# plt.xticks(x, raw_sent[0], rotation=45)
# plt.bar(x, y)
# plt.ylabel('%')
# plt.title('Visualisation of words importance')
# plt.show()

In [ ]:
loaded_model = LMClassifierWrapper(from_fp='bert_en_sentiment.bin')
loaded_model.init_model()

print(loaded_model(test_item))

In [ ]:
classifier(test_item, return_logits=True)[0]

In [ ]:
print(loaded_model.__getstate__()['state_dict'])

In [ ]:
print(classifier.__getstate__()['state_dict'])

In [ ]:
def compare_models(model_1, model_2):
    models_differ = 0
    for key_item_1, key_item_2 in zip(model_1.state_dict().items(), model_2.state_dict().items()):
        if torch.equal(key_item_1[1], key_item_2[1]):
            pass
        else:
            models_differ += 1
            if (key_item_1[0] == key_item_2[0]):
                print('Mismtach found at', key_item_1[0])
            else:
                raise Exception
    if models_differ == 0:
        print('Models match perfectly! :)')


In [ ]:
compare_models(loaded_model.model, classifier.model)

In [ ]:
loaded_model.featurizer.transform(test_item)

In [ ]:
X_train = classifier.featurizer.transform(test_item).cuda()

In [ ]:
raw_loaded_model = loaded_model.model
raw_classifier = classifier.model

In [ ]:
raw_loaded_model.eval()
raw_classifier.eval()
raw_loaded_model.encoder = raw_classifier.encoder
raw_loaded_model(X_train)[0] - raw_classifier(X_train)[0]

In [ ]:
from common.torch_utils import cauchy
raw_loaded_model.eval()
raw_classifier.eval()

encoded = raw_loaded_model.encoder(X_train)[1]
rnn = raw_loaded_model.rnn(encoded)[0]
max_pool = torch.max(rnn, 1)[0]
pooled = max_pool
prev_pooled = max_pool
# print(prev_pooled - pooled)
# pooled = raw_loaded_model.pooler(max_pool)
for ix, layer in enumerate(raw_loaded_model.pooler):
    pooled = layer(pooled)
    prev_pooled = raw_classifier.pooler[ix](prev_pooled)
#     print(ix)
#     print(prev_pooled - pooled)
#     print(layer.weight - raw_classifier.pooler[ix].weight)
#     print(layer.bias - raw_classifier.pooler[ix].bias)
#     print(ix)
#     print(len(raw_loaded_model.pooler))
#     print(pooled - prev_pooled)
    if ix < len(raw_loaded_model.pooler) - 1:
        pooled = torch.nn.functional.relu(pooled)
        prev_pooled = torch.nn.functional.relu(prev_pooled)
#         print(prev_pooled - pooled)
#         print(pooled)
    else:
        pooled = cauchy(pooled)
        prev_pooled = cauchy(prev_pooled)
#         print(prev_pooled - pooled)
#         print(pooled)

# print(pooled - prev_pooled)
print(raw_loaded_model.classifier.bias)
print(raw_classifier.classifier.bias)
with torch.no_grad():
    print(raw_loaded_model.classifier.weight - raw_classifier.classifier.weight)
    print(raw_loaded_model.classifier(prev_pooled))
    print(raw_classifier.classifier(prev_pooled))

In [ ]:
print(max_pool)

In [ ]:
print(loaded_model.model.encoder == raw_loaded_model.encoder)
print(loaded_model(test_item))